In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

## Loading the dataset containing the neighborhoods of Delhi


In [34]:
df_delhi=pd.read_csv("Dataset containing Neighborhoods in Delhi.csv")

In [35]:
df_delhi.head()

,Unnamed: 0,Location,Pincode,State,District,Latitude,Longitude
0,Nan,Ashok Vihar,110052,Delhi,North West Delhi,28.68839,77.68839
1,Nan,Bawana,110039,Delhi,North West Delhi,28.80000,77.03000
2,Nan,Begum Pur,110086,Delhi,North West Delhi,28.72714,77.05948
3,Nan,Dhaka,110052,Delhi,North West Delhi,28.70780,77.20570
4,Nan,Karala,110081,Delhi,North West Delhi,28.73800,77.03920


In [36]:
df_delhi.drop('Unnamed: 0', axis=1, inplace=True)
df_delhi.head()

,Location,Pincode,State,District,Latitude,Longitude
0,Ashok Vihar,110052,Delhi,North West Delhi,28.68839,77.68839
1,Bawana,110039,Delhi,North West Delhi,28.80000,77.03000
2,Begum Pur,110086,Delhi,North West Delhi,28.72714,77.05948
3,Dhaka,110052,Delhi,North West Delhi,28.70780,77.20570
4,Karala,110081,Delhi,North West Delhi,28.73800,77.03920


## Finding the latitude and longitude of Delhi


In [37]:
address = 'Delhi'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi are 28.6517178, 77.2219388.


## Creating a map

## Foursquare Credentials and Version

In [38]:
CLIENT_ID = 'KGJ0LDNRT2WDSKMUTW1GJKBDYJB123T5ELYY03RB5QGZKJND' # your Foursquare ID
CLIENT_SECRET = 'R0G5LO0W0UEDVGX5SADFFWV4FZJJBHGCCJYSG242W33S2VZY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KGJ0LDNRT2WDSKMUTW1GJKBDYJB123T5ELYY03RB5QGZKJND
CLIENT_SECRET:R0G5LO0W0UEDVGX5SADFFWV4FZJJBHGCCJYSG242W33S2VZY


## Getting Nearby Venues

In [39]:
radius=500
LIMIT=100
venues_list=[]
    
for lat,lng,loca,post in zip(df_delhi['Latitude'], df_delhi['Longitude'], df_delhi['Location'], df_delhi['Pincode']):
            
        
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
    results = requests.get(url).json()["response"]['groups'][0]['items']
        
    for v in results:
        venues_list.append((
            post, 
            loca,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']))

In [40]:
df_with_venues = pd.DataFrame(venues_list)
df_with_venues.columns = ['Pincode', 'Location', 'Latitude', 'Longitude', 'Venue_Name', 'Venue_Latitude', 'Venue_Longitude', 'Venue_Category']
df_with_venues.head()

,Pincode,Location,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,110039,Bawana,28.8000,77.0300,Rajiv Gandhi Stadium,28.801811,77.028751,Playground
1,110052,Dhaka,28.7078,77.2057,Pizza Hut,28.706996,77.204623,Pizza Place
2,110052,Dhaka,28.7078,77.2057,Gtb Nagar Railway Station,28.707183,77.206757,Train Station
3,110035,Keshav Puram,28.6889,77.1616,Major Dhyanchand Sports Complex.,28.689482,77.163528,Basketball Court
4,110035,Keshav Puram,28.6889,77.1616,"Batra, The designer house",28.687630,77.161940,Women's Store


## Number of venues per groups

In [41]:
df_with_venues.groupby("Location").count()


,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
Location,,,,,,,
Ashok Nagar,4,4,4,4,4,4,4
Babarpur,3,3,3,3,3,3,3
Badarpur,1,1,1,1,1,1,1
Bali Nagar,6,6,6,6,6,6,6
Bawana,1,1,1,1,1,1,1
Chanakyapuri,21,21,21,21,21,21,21
Chandni Chowk,14,14,14,14,14,14,14
Chawri Bazar,11,11,11,11,11,11,11
Chittaranjan Park,1,1,1,1,1,1,1


## Check if there are Indian Restaurant in the Venue Categories


In [42]:
"Indian Restaurant" in df_with_venues['Venue_Category'].unique()


True

 ## Analyzing Locations

In [43]:
del_onehot = pd.get_dummies(df_with_venues[['Venue_Category']], prefix="", prefix_sep="")
del_onehot['Location'] = df_with_venues['Location']
fixed_columns = [del_onehot.columns[-1]] + list(del_onehot.columns[:-1])
del_onehot = del_onehot[fixed_columns]
del_onehot.head()

,Location,ATM,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Basketball Court,Bengali Restaurant,Bistro,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Event Space,Fabric Shop,Fast Food Restaurant,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Miscellaneous Shop,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Neighborhood,Nightclub,Northeast Indian Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Restaurant,Road,Sandwich Place,Sculpture Garden,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Stadium,Theater,Theme Restaurant,Tibetan Restaurant,Train Station,Vietnamese Restaurant,Women's Store
0,Bawana,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dhaka,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dhaka,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Keshav Puram,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Keshav Puram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## Group rows by location and by taking the mean of the frequency of occurrence of each category

In [44]:
del_grouped = del_onehot.groupby(['Location']).mean().reset_index()
del_grouped.head()


,Location,ATM,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Basketball Court,Bengali Restaurant,Bistro,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Event Space,Fabric Shop,Fast Food Restaurant,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Miscellaneous Shop,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Neighborhood,Nightclub,Northeast Indian Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Restaurant,Road,Sandwich Place,Sculpture Garden,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Stadium,Theater,Theme Restaurant,Tibetan Restaurant,Train Station,Vietnamese Restaurant,Women's Store
0,Ashok Nagar,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,Babarpur,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Badarpur,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,Bali Nagar,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0
4,Bawana,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## Only Indian Restaurant

In [45]:
del_playground = del_grouped[['Location', 'Indian Restaurant']]
del_playground.head()

,Location,Indian Restaurant
0,Ashok Nagar,0.0
1,Babarpur,0.0
2,Badarpur,0.0
3,Bali Nagar,0.0
4,Bawana,0.0


## Clustering

In [46]:
kclusters = 4
del_cluster = del_playground.drop('Location', axis=1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(del_cluster)
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 1, 2, 0, 3], dtype=int32)

## Creating new dataframe

In [47]:
del_merged = del_playground.copy()
del_merged['Cluster Labels'] = kmeans.labels_
del_merged = del_merged.join(df_with_venues.set_index('Location'), on='Location')

In [48]:
del_merged.head()


,Location,Indian Restaurant,Cluster Labels,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,Ashok Nagar,0.0,0,110018,28.637267,77.10385,Giani Ice Cream,28.638913,77.101122,Ice Cream Shop
0,Ashok Nagar,0.0,0,110018,28.637267,77.10385,Subhash Nagar Metro station,28.640385,77.105252,Light Rail Station
0,Ashok Nagar,0.0,0,110018,28.637267,77.10385,"Versus, Pacific Mall",28.641092,77.104902,Bowling Alley
0,Ashok Nagar,0.0,0,110018,28.637267,77.10385,Miraj Cinemas,28.639240,77.108174,Multiplex
1,Babarpur,0.0,0,110032,28.687730,77.28044,Shree Balaji Handicrafts,28.688084,77.282681,Gift Shop


## Visualizing Clusters

In [49]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, loca, cluster in zip(del_merged['Latitude'],del_merged['Longitude'], del_merged['Location'], del_merged['Cluster Labels']):
    label = folium.Popup(str(loca) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

In [50]:
del_merged.loc[(del_merged['Cluster Labels'] == 0) & (del_merged['Venue_Category'] == 'Indian Restaurant ')]

,Location,Indian Restaurant,Cluster Labels,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category


In [51]:
del_merged.loc[(del_merged['Cluster Labels'] == 1) & (del_merged['Venue_Category'] == 'Indian Restaurant')]


,Location,Indian Restaurant,Cluster Labels,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
6,Chandni Chowk,0.357143,1,110006,28.656000,77.231000,Parawthe Wale,28.656033,77.230500,Indian Restaurant
6,Chandni Chowk,0.357143,1,110006,28.656000,77.231000,Babu Ram Paranthe Wale,28.655820,77.230448,Indian Restaurant
6,Chandni Chowk,0.357143,1,110006,28.656000,77.231000,Pt. Kanhaiya Lal Durga Prashad Dixit Parawthe ...,28.656109,77.230465,Indian Restaurant
6,Chandni Chowk,0.357143,1,110006,28.656000,77.231000,Pt. Gaya Prasad Shiv Charan Paranthewale | पंड...,28.656027,77.230438,Indian Restaurant
6,Chandni Chowk,0.357143,1,110006,28.656000,77.231000,Old Famous Jalebi Waala,28.656012,77.233604,Indian Restaurant
11,Dariba Kalan,0.357143,1,110054,28.655300,77.234200,Parawthe Wale,28.656033,77.230500,Indian Restaurant
11,Dariba Kalan,0.357143,1,110054,28.655300,77.234200,Babu Ram Paranthe Wale,28.655820,77.230448,Indian Restaurant
11,Dariba Kalan,0.357143,1,110054,28.655300,77.234200,Pt. Kanhaiya Lal Durga Prashad Dixit Parawthe ...,28.656109,77.230465,Indian Restaurant
11,Dariba Kalan,0.357143,1,110054,28.655300,77.234200,Pt. Gaya Prasad Shiv Charan Paranthewale | पंड...,28.656027,77.230438,Indian Restaurant
11,Dariba Kalan,0.357143,1,110054,28.655300,77.234200,Old Famous Jalebi Waala,28.656012,77.233604,Indian Restaurant


In [52]:
del_merged.loc[(del_merged['Cluster Labels'] == 2) & (del_merged['Venue_Category'] == 'Indian Restaurant')]

,Location,Indian Restaurant,Cluster Labels,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
7,Chawri Bazar,0.272727,2,110006,28.650402,77.229379,Karim's | करीम | کریم (Karim's),28.649498,77.233691,Indian Restaurant
7,Chawri Bazar,0.272727,2,110006,28.650402,77.229379,Al Jawahar | अल जवाहर | امام جواہر,28.649609,77.233555,Indian Restaurant
7,Chawri Bazar,0.272727,2,110006,28.650402,77.229379,Aslam Chicken Corner,28.648283,77.233648,Indian Restaurant
15,Delhi Cantonment,0.250000,2,110010,28.596000,77.130000,Turning point,28.595783,77.131773,Indian Restaurant
20,Gole Market,0.200000,2,110001,28.633700,77.205600,Sarvam Safari,28.632028,77.207447,Indian Restaurant
20,Gole Market,0.200000,2,110001,28.633700,77.205600,Chutney,28.631367,77.208054,Indian Restaurant
21,Govind Puri,0.250000,2,110019,28.535400,77.263900,Gupta Kachori Stall,28.537866,77.260785,Indian Restaurant
27,Kailash Colony,0.200000,2,110048,28.544400,77.232550,Kake Da Hotel,28.542149,77.231666,Indian Restaurant
35,Munirka,0.250000,2,110023,28.553212,77.174482,"Udupi Restaurant, Munirka",28.555178,77.176247,Indian Restaurant
38,Netaji Nagar,0.230769,2,110023,28.582080,77.194040,Tamil Nadu House Canteen,28.583112,77.191033,Indian Restaurant


In [53]:
del_merged.loc[(del_merged['Cluster Labels'] == 3) & (del_merged['Venue_Category'] == 'Indian Restaurant')]

,Location,Indian Restaurant,Cluster Labels,Pincode,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
9,Civil Lines,0.090909,3,110054,28.679368,77.226076,The Embassy,28.676556,77.224969,Indian Restaurant
14,Defence Colony,0.148148,3,110024,28.570000,77.230000,Sagar Ratna,28.572839,77.230604,Indian Restaurant
14,Defence Colony,0.148148,3,110024,28.570000,77.230000,Swagath Restaurant & Bar,28.573489,77.230183,Indian Restaurant
14,Defence Colony,0.148148,3,110024,28.570000,77.230000,Colonel's Kababz,28.574009,77.230490,Indian Restaurant
14,Defence Colony,0.148148,3,110024,28.570000,77.230000,Liquid Kitchen,28.573907,77.230313,Indian Restaurant
36,Naraina,0.111111,3,110023,28.632141,77.138855,Bikanervala Naraina,28.630711,77.138122,Indian Restaurant
45,Sadar Bazar,0.166667,3,110006,28.658813,77.216742,Md. Hussain Fried Chicken,28.658860,77.220394,Indian Restaurant


## Cluster 0 